In [1]:
# New Tricks Data Analysis
# ------------------------

# Assignment
# ----------
# The attached .txt files contain client registration data collected by New Tricks, 
# a wildly popular (and fictitious) network of canine training academies located in the New York City area. 
# Their legacy registration system has limited reporting capabilities and can only export records in a very raw format. 
# The New Tricks marketing team wants to use their historical client data 
# to inform future promotional campaigns, so they've hired you to help interpret it!

# Objectives
# ----------
# 1. The team wants to advertise the total number of students served on the front page of their website. 
# They were hoping to add up the total in their legacy exports but weren't sure how to work with the format. 
# It also looked like the same students were sometimes showing up more than once. Please provide the following:

# a) Calculate the total number of canine students.
# b) Describe how you dealt with the format of the files.
# c) Explain how you arrived at the total.

# 2. Next, the team is looking to kick off a campaign promoting the newly reduced rate for their Gold-level course package. 
# They want to reach out to the owners of all students who have recently registered for a Gold-Level package to see 
# if they'd be interested in being featured in the campaign.

# Please use the information in the exports to create a list of all of the students who have registered for a Gold Package since 1 January 2019. 
# They just need to know the students' unique ID numbers so they can look up their contact info in the system. 
# You can attach the list in any format.

# 3. New Tricks also wants to grow the size of their mixed breed client base. 
# To that end, they'd like to know which branch areas are seeing the least amount of mixed-breed students coming in.

# Please use the information in the exports along with the zip code region ranges table to 
# determine which two regions have the smallest percentage of registrations by students classified in the system as "Mixed/Other". 
# They'd also like to know what methods you used to come up with the answer. 

# 4. Lastly, the team isn't entirely confident in the capabilities of their legacy system, 
# so they just want to know if you think they can trust the information in these exports. 
# Based on your work with the files, did you notice any red flags that might cause you to doubt the integrity of the data? 
# Any specific details you can provide to back your answer should help to put them at ease!


In [2]:
# Step 1: Import the libraries
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
# Step 2: Read the dataset: Registrations
dfregs = pd.read_csv('/Users/Mac/Desktop/Greenhouse/new_tricks/registrations.csv')

# Clean the header columns
dfregs.columns = dfregs.columns.str.replace('[<,>]', '')

# Clean the records
dfregs = dfregs.replace({'<':'', '>':''}, regex=True)

dfregs.head()

,ID,package,registrant_ID,registration_date
0,239,Standard,788900239,8262007
1,240,Free Trial,788900240,5272006
2,241,Standard,788900241,9012006
3,242,Standard,788900242,6101995
4,243,Standard,788900243,2191997


In [4]:
# Remove duplicate entries
dfregsdrop = dfregs.drop_duplicates()

dfregsdrop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81539 entries, 0 to 81538
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ID                 81539 non-null  object
 1   package            81539 non-null  object
 2   registrant_ID      81539 non-null  object
 3   registration_date  81539 non-null  object
dtypes: object(4)
memory usage: 3.1+ MB


In [5]:
# Convert columns from object to integer
dfregs['ID'] = dfregs['ID'].astype(int)

dfregs['registrant_ID'] = dfregs['registrant_ID'].astype(int)

dfregs['registration_date'] = dfregs['registration_date'].astype(int)

In [6]:
# Step 2a: Read the dataset: Registrant Profiles
dfpro = pd.read_csv('/Users/Mac/Desktop/Greenhouse/new_tricks/registrant_profiles.csv')

# Clean the header columns
dfpro.columns = dfpro.columns.str.replace('[<,>]', '')

# Clean the records
dfpro = dfpro.replace({'<':'', '>':''}, regex=True)

# Convert records to proper case
dfpro['dog_name'] = dfpro['dog_name'].str.title()

dfpro['dominant_color'] = dfpro['dominant_color'].str.title()

dfpro['secondary_color'] = dfpro['secondary_color'].str.title()

dfpro.head()

,registrant_ID,dog_name,gender,breed,dob,dominant_color,secondary_color,zip_code
0,788905773,Shakespere,M,Labrador Retriever Crossbreed,41066,White,Tan,10308
1,788917912,Pickles,F,Havanese,41376,White,Black,10128
2,788956064,Butter,M,German Shepherd Dog,41068,Black,Tan,10304
3,788974553,Pira,F,Mixed/Other,35584,Black,Gray,11218
4,788917962,Max,F,Yorkshire Terrier,41104,Black,Brown,11237


In [7]:
# Review data types for the purpose of analysis
dfpro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80537 entries, 0 to 80536
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   registrant_ID    80537 non-null  object
 1   dog_name         80537 non-null  object
 2   gender           80537 non-null  object
 3   breed            80537 non-null  object
 4   dob              80537 non-null  object
 5   dominant_color   80537 non-null  object
 6   secondary_color  80537 non-null  object
 7   zip_code         80537 non-null  object
dtypes: object(8)
memory usage: 4.9+ MB


In [8]:
# Remove duplicate entries
dfprodrop = dfpro.drop_duplicates()

dfprodrop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80537 entries, 0 to 80536
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   registrant_ID    80537 non-null  object
 1   dog_name         80537 non-null  object
 2   gender           80537 non-null  object
 3   breed            80537 non-null  object
 4   dob              80537 non-null  object
 5   dominant_color   80537 non-null  object
 6   secondary_color  80537 non-null  object
 7   zip_code         80537 non-null  object
dtypes: object(8)
memory usage: 5.5+ MB


In [9]:
# Review missing value rows before conversion of data type
dfpro[dfpro['zip_code'] == '']

,registrant_ID,dog_name,gender,breed,dob,dominant_color,secondary_color,zip_code
18183,#N/A,,,,520,,,


In [10]:
# Drop the non-value record
dfpro = dfpro.drop(18183)

In [11]:
# Clean the records
dfpro = dfpro.replace({'#':''}, regex=True)

dfpro = dfpro.replace({'N/A':'-'}, regex=True)

In [12]:
# Convert columns from object to integer
dfpro['registrant_ID'] = dfpro['registrant_ID'].astype(int)

dfpro['dob'] = dfpro['dob'].astype(int)

dfpro['zip_code'] = dfpro['zip_code'].astype(int)

In [13]:
# Step 2b: Read the dataset: Zipcode Region Ranges
dfzip = pd.read_csv('/Users/Mac/Desktop/Greenhouse/new_tricks/zipcode_region_ranges.csv')

# Clean the header columns
dfzip.columns = dfzip.columns.str.replace('[<,>]', '')

# Clean the records
dfzip = dfzip.replace({'<':'', '>':''}, regex=True)

dfzip['min_zip_code'] = dfzip['min_zip_code'].astype(int)

dfzip['max_zip_code'] = dfzip['max_zip_code'].astype(int)

dfzip.head(6)

,region_name,min_zip_code,max_zip_code
0,Manhattan,10001,10282
1,Staten Island,10301,10314
2,Bronx,10451,10475
3,Queens,11004,11106
4,Brooklyn,11201,11249
5,Queens,11351,11697


In [14]:
dfzip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   region_name   6 non-null      object
 1   min_zip_code  6 non-null      int64 
 2   max_zip_code  6 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 272.0+ bytes


In [15]:
# Step 3: Uncover insights
# Question 1:
# 1. The team wants to advertise the total number of students served on the front page of their website. 
# They were hoping to add up the total in their legacy exports but weren't sure how to work with the format. 
# It also looked like the same students were sometimes showing up more than once. Please provide the following:

# a) Calculate the total number of canine students.
# b) Describe how you dealt with the format of the files.
# c) Explain how you arrived at the total.

In [16]:
# Question 1a: Calculate the total number of canine students

# Check if Registrant ID is the same for both datasets
dfregs['registrant_ID'].isin(dfpro['registrant_ID']).value_counts()

True    81539
Name: registrant_ID, dtype: int64

In [17]:
total_students = dfregs['ID'].nunique()
print('The total number of canine students is {:,}.'.format(81539))

The total number of canine students is 81,539.


Question 1b: Describe how you dealt with the format of the files.

- To work with the files and provide analysis, it was necessary to convert each of the legacy exports to .csv format for reading into Python. Microsoft Excel was used in the process of conversion. The integral step of analysis, when working with .csv files, is the delimiter that allows for spacing within the data for easier readability. With the .csv file, all of the information is organized to be used by most programming or visualization tools. Once the conversion was complete, I brought the files into Python for analysis.

Question 1c: Explain how you arrived at the total.

- In order to find the total number of canine students I first asked: which export contains the primary key for students registrations? Both Registrations and Registrant Profiles contain student information. However, given the description of the files it was clear the Registrations file was the foundation for the other exports. Within the Registrations file is the Registrant_ID column. This column informs me that every student in the system has completed the onboarding process and has been assigned an ID. This column contains the primary key that is found in the Registrant Profiles export. 


- With the Registrant ID, I checked for duplicate records and did not find any within the export. Once I knew there weren't any duplicates I compared them to the records in the Registrant Profiles export and there was a match. To check against this I found the count of unique ID's to be the same as total number of registrations.

In [18]:
# Question 2:
# Next, the team is looking to kick off a campaign promoting the newly reduced rate for their Gold-level course package. 
# They want to reach out to the owners of all students who have recently registered for a Gold-Level package to see 
# if they'd be interested in being featured in the campaign.

# Please use the information in the exports to create a list of 
# all of the students who have registered for a Gold Package since 1 January 2019. 
# They just need to know the students' unique ID numbers so they can look up their contact info in the system. 
# You can attach the list in any format.

# Convert column from integer to date
dfregs['registration_date'] = pd.to_datetime(dfregs['registration_date'], format='%m%d%Y')

# Create variable for Gold package and Registration Date
gold_pack = (dfregs['package'] == 'Gold') & (dfregs['registration_date'] >= '2019-01-01')

# Create variable for single column output
yearjan = dfregs.iloc[gold_pack.values,[0]]

#print
yearjan

,ID
13047,66571
16463,23237
20248,4559
24494,17777
34230,43810
41758,61311
42969,21917
48596,78686
57948,62891
61199,54903


- Result: 14 students have registered for a Gold package since January 1st, 2019.

In [19]:
# Question 3:
# New Tricks also wants to grow the size of their mixed breed client base. 
# To that end, they'd like to know which branch areas are seeing the least amount of mixed-breed students coming in.

## Please use the information in the exports along with the zip code region ranges table to 
# determine which two regions have the smallest percentage of registrations by 
# students classified in the system as "Mixed/Other". 
# They'd also like to know what methods you used to come up with the answer. 

# Define function for Region
def region(zip_code):
    if 10001 >= zip_code <= 10282:
        return "Manhattan"
    if 10301 >= zip_code <= 10314:
        return "Staten Island"
    if 10451 >= zip_code <= 10475:
        return "Bronx"
    if 11004 >= zip_code <= 11106:
        return "Queens"
    if 11201 >= zip_code <= 11249:
        return "Brooklyn"
    else:
        return "-"
    return region

# Add new column
dfpro['new_region'] = dfpro['zip_code'].apply(region)

dfpro.head()

,registrant_ID,dog_name,gender,breed,dob,dominant_color,secondary_color,zip_code,new_region
0,788905773,Shakespere,M,Labrador Retriever Crossbreed,41066,White,Tan,10308,Bronx
1,788917912,Pickles,F,Havanese,41376,White,Black,10128,Staten Island
2,788956064,Butter,M,German Shepherd Dog,41068,Black,Tan,10304,Bronx
3,788974553,Pira,F,Mixed/Other,35584,Black,Gray,11218,-
4,788917962,Max,F,Yorkshire Terrier,41104,Black,Brown,11237,-


In [20]:
# Create variable for Mixed/Other value
mixed = dfpro['breed'] == 'Mixed/Other'

In [21]:
# Get the distribution of Region
dfpro.groupby(mixed)['new_region'].value_counts(normalize=True)[10:12]

breed  new_region
True   Brooklyn      0.043623
       Manhattan     0.005857
Name: new_region, dtype: float64

- Result: 
The two regions that have the smallest percentage of registrations are Brooklyn with 4.3% and Manhattan with .05%.

Question 4: Lastly, the team isn't entirely confident in the capabilities of their legacy system, so they just want to know if you think they can trust the information in these exports. Based on your work with the files, did you notice any red flags that might cause you to doubt the integrity of the data? Any specific details you can provide to back your answer should help to put them at ease!

- The team can feel confidence in working with their legacy system. Based on my analysis there aren't any duplicate records in the two of the three exports. The duplication in the third export helped to identify another set of zip codes for a region. Also, there is a wealth of information about students and their profiles to build out the details of important business problems.


- The team at New Tricks has data in their exports to find answers about the business but the legacy system will last only as long as it is maintained. Unfortunately, long-term maintenance of this information can be costly and the ever-changing technology lanscape demands modern practices. The services that our teams provide proof that your information is secure, follows the best practices in this modern age and most importantly, can be maintained effectively to further the goals of the business.  
